In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
%load_ext line_profiler # Example: %lprun -T lprof0 -f celeri.celeri.get_okada_displacements u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1.values[segment_idx], segment.lat1[segment_idx], segment.lon2[segment_idx], segment.lat2[segment_idx], segment.locking_depth[segment_idx], segment.burial_depth[segment_idx], segment.dip[segment_idx], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)

In [ ]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [ ]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
celeri.plot_block_labels(segment, block, station, closure)

In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
operators.block_rotation = celeri.get_block_rotation_operator(station, block)
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment) # This function need work
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)
block_strain_rate_operator, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
celeri.plot_strain_rate_components_for_block(closure, segment, station, block_idx=0)

In [142]:
a = station.to_dict()

In [175]:
# def get_block_rotation_operator(station, block):
#     """
#     Get partial derivatives relating displacement to Euler pole rotation
#     TODO: Do I have to reoder the blocks dataframe so that block label 0 is at the top?
#     If so this should be done right after the block closure algorithm is run rather than here.
#     """
#     block_rotation_operator = np.zeros((3 * len(station), 3 * len(block)))
#     for i in range(len(station)):
#         start_row_idx = 3 + i
#         start_column_idx = 3 * station.block_label[i]
#         cross_product_operator = celeri.get_cross_partials(
#             [station.x[i], station.y[i], station.z[i]]
#         )
#         vn_wx, ve_wx, vu_wx = celeri.cartesian_vector_to_spherical_vector(
#             cross_product_operator[0, 0],
#             cross_product_operator[1, 0],
#             cross_product_operator[2, 0],
#             station.lon[i],
#             station.lat[i],
#         )
#         vn_wy, ve_wy, vu_wy = celeri.cartesian_vector_to_spherical_vector(
#             cross_product_operator[0, 1],
#             cross_product_operator[1, 1],
#             cross_product_operator[2, 1],
#             station.lon[i],
#             station.lat[i],
#         )
#         vn_wz, ve_wz, vu_wz = celeri.cartesian_vector_to_spherical_vector(
#             cross_product_operator[0, 2],
#             cross_product_operator[1, 2],
#             cross_product_operator[2, 2],
#             station.lon[i],
#             station.lat[i],
#         )
#         block_rotation_operator[
#             start_row_idx : start_row_idx + 3, start_column_idx : start_column_idx + 3
#         ] = np.array(
#             [[ve_wx, ve_wy, ve_wz], [vn_wx, vn_wy, vn_wz], [vu_wx, vu_wy, vu_wz]]
#         )
#     return block_rotation_operator

# TODO: Function to take rotation vector and predict enu velocities on a block or at all stations


def get_rotation_displacements(station, omega_x, omega_y, omega_z):
    """
    Get displacments at all stations
    """
    vel_east = np.zeros(len(station))
    vel_north = np.zeros(len(station))
    vel_up = np.zeros(len(station))
    for i in range(len(station)):
        cross_product_operator = celeri.get_cross_partials([station.x.values[i], station.y.values[i], station.z.values[i]])
        vel_north_from_omega_x, vel_east_from_omega_x, vel_up_from_omega_x = celeri.cartesian_vector_to_spherical_vector(cross_product_operator[0, 0], cross_product_operator[1, 0], cross_product_operator[2, 0], station.lon.values[i], station.lat.values[i])
        vel_north_from_omega_y, vel_east_from_omega_y, vel_up_from_omega_y = celeri.cartesian_vector_to_spherical_vector(cross_product_operator[0, 1], cross_product_operator[1, 1], cross_product_operator[2, 1], station.lon.values[i], station.lat.values[i])
        vel_north_from_omega_z, vel_east_from_omega_z, vel_up_from_omega_z = celeri.cartesian_vector_to_spherical_vector(cross_product_operator[0, 2], cross_product_operator[1, 2], cross_product_operator[2, 2], station.lon.values[i], station.lat.values[i])
        vel_east[i] = omega_x * vel_east_from_omega_x + omega_y * vel_east_from_omega_y + omega_z * vel_east_from_omega_z
        vel_north[i] = omega_x * vel_north_from_omega_x + omega_y * vel_north_from_omega_y + omega_z * vel_north_from_omega_z
    return vel_east, vel_north, vel_up


block_idx = 4
vel_east, vel_north, vel_up = get_rotation_displacements(station[station.block_label==block_idx], omega_x=1, omega_y=0, omega_z=0)

plt.figure()
plt.plot(vel_east, "-r")
plt.plot(vel_north, "-b")
plt.plot(vel_up, "-g")
plt.show()
print(len(station))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1686


In [176]:


def plot_strain_rate_components_for_block(closure, station, block_idx):
    plt.figure(figsize=(10, 3))
    plt.subplot(1, 3, 1)
    vel_east, vel_north, vel_up = get_rotation_displacements(
        station,
        omega_x=1,
        omega_y=0,
        omega_z=0,
    )
    for i in range(closure.n_polygons()):
        plt.plot(
            closure.polygons[i].vertices[:, 0],
            closure.polygons[i].vertices[:, 1],
            "k-",
            linewidth=0.5,
        )
    plt.quiver(
        station.lon,
        station.lat,
        vel_east,
        vel_north,
        scale=1e7,
        scale_units="inches",
        color="r",
    )

    plt.subplot(1, 3, 2)
    vel_east, vel_north, vel_up = get_rotation_displacements(
        station,
        omega_x=0,
        omega_y=1,
        omega_z=0,
    )
    for i in range(closure.n_polygons()):
        plt.plot(
            closure.polygons[i].vertices[:, 0],
            closure.polygons[i].vertices[:, 1],
            "k-",
            linewidth=0.5,
        )
    plt.quiver(
        station.lon,
        station.lat,
        vel_east,
        vel_north,
        scale=1e7,
        scale_units="inches",
        color="r",
    )

    plt.subplot(1, 3, 3)
    vel_east, vel_north, vel_up = get_rotation_displacements(
        station,
        omega_x=0,
        omega_y=0,
        omega_z=1,
    )
    for i in range(closure.n_polygons()):
        plt.plot(
            closure.polygons[i].vertices[:, 0],
            closure.polygons[i].vertices[:, 1],
            "k-",
            linewidth=0.5,
        )
    plt.quiver(
        station.lon,
        station.lat,
        vel_east,
        vel_north,
        scale=1e7,
        scale_units="inches",
        color="r",
    )
    plt.show()

plot_strain_rate_components_for_block(closure, station, block_idx)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [156]:
len(a)
len(station)

1686